In [1]:
import numpy as np

We want to implement the Binomial Asset Pricing Model and use it to price derivative securities. We will employ two methods: the first is a Monte Carlo method in which we compute the risk neutral probabilities and then generate one random path, and compute its discounted price. The second is to implement a binomial tree, compute the value of the option at expiry, and then use that information to work backwards up the tree to arrive at the present value of the option. The MC method both in implementation and precision gives a quick and dirty answer, it is easy to write the code, and for moderate trial sizes is very quick, however the answers, as with all MC based methods have high accuracy, low precision. The binomial tree implementation is a bit more involved, but gives exact answers, and for modest tree sizes is not much slower than the MC method, but is much more precise.

The relevant variables you may see referenced in function defintions are as follows: S0 is the asset price at time 0, r is the risk-free rate, K is the strike price, u is the "up" factor (the rate at which the price of the asset increases after one period), pu is the risk-neutral probability of selecting up, d is the "down" factor (the rate at which the price of the asset decreases after one period), pd is the risk-neutral probability of picking down, and n is the number of periods in the model.

In [197]:
def BAMP_MC(S0,r,K,u,d,n):
    # Compute the risk neutral probabilities
    pu = (1+r-d)/(u-d)
    pd = (u-1-r)/(u-d)
    
    # Choose paths randomly n times, compute the discounted price at expiry
    for i in range(0,n):
    #This statement picks from u with probability pu, and d with probability pd
        S0 *= np.random.choice([u,d],p=[pu,pd])
    deriv_value = (1/(1+r))**n * max(S0-K,0)
    return deriv_value     

We now take our MC method to generate one path, and average over some large number of paths, N.

In [136]:
def BAMP_MC_Opt_Price(S0,r,K,u,d,n,N):
    opt_value=0
    for i in range(1,N):
        opt_value += BAMP_MC(S0,r,K,u,d,n)
    opt_value /= N
    return opt_value    

We will now check our model against a known result. This happens to be the first example computed in Shreve, and we should get V0 = 1.2

In [18]:
BAMP_MC_Opt_Price(4,.25,5,2,0.5,1,10000)

1.2062399999998903

The binomial tree implementation will proceed as follows: We define a node class, each node has a value and two children, which are inititally assigned to None. We will then construct a binomial or full tree (A full tree is one in which each level is maximally populated, a full tree of height h has 2^h - 1 nodes.) We do this by iteratively adding layers to the tree by iterating over a list which stores the set of nodes.

In [198]:
class Node:
    def __init__(self,val,depth):
        self.left = None
        self.right = None
        self.value = val
        self.depth = depth

In [95]:
class BinomialTree:
    
    # This constructor takes in the initial asset price, and adds the root as the first element of the list of nodes 
    def __init__(self,S):
        self.root = Node(S,0)
        self.node_list = list()
        self.node_list.append(self.root)
    
    def getRoot(self):
        return self.root
    
    # This method adds both left and right children with the appropriate up and down factor
    # Our convention is that the left child is the down factor, and the right child is the up factor
    def add_children(self,node,u,d):
        if(node.left == None and node.right == None):
            node.left = Node(d*node.value, node.depth+1)
            self.node_list.append(node.left)
            node.right = Node(u*node.value, node.depth+1)
            self.node_list.append(node.right)
        else:
            pass
    
    # This method adds a layer by iterating through the list, since the add children method passes over already
    # populated nodes it can safely iterate the entire list. Note: There is certainly a faster way using
    # the depth characteristic and list comprehensions, but this works for now
    def add_layer(self,u,d):
        for i in range(0,len(self.node_list)):
            self.add_children(self.node_list[i],u,d)
    
    # This iteratively adds layers
    def pop_full_tree(self,u,d,max_depth):
        for i in range(0,max_depth):
            self.add_layer(u,d)
            
    #This was mostly used for debugging so I could check that my trees were functioning properly    
    def print_tree(self):
        for i in range(0,len(self.node_list)):
            print self.node_list[i].value
    

With the tree implementation in place we now can price our option. To do this we employ the following method. The logic is as follows: construct two trees, one of which stores the prices of assets, the other reads the price at expiry from the first tree and computes the derivative price. It then works backwards up the tree using risk neutral pricing to compute the price of the derivative security at each time step.

In [185]:
def BAMP_Tree_Opt_Price(S0,r,K,u,d,n):
    pu = (1+r-d)/(u-d)
    pd = (u-1-r)/(u-d)
    asset_tree = BinomialTree(S0)
    asset_tree.pop_full_tree(u,d,n)
    deriv_tree = BinomialTree(0)
    deriv_tree.pop_full_tree(0,0,n)
    
    for i in range(2**n-1,len(deriv_tree.node_list)):
        deriv_tree.node_list[i].value = max(asset_tree.node_list[i].value-K,0)
    
    for i in range(0,(len(deriv_tree.node_list)-2**n)):
        deriv_tree.node_list[(len(deriv_tree.node_list)-1-2**n)-i].value = (1/(1+r))*((pd*deriv_tree.node_list[(len(deriv_tree.node_list)-1-2**n)-i].left.value) + (pu*deriv_tree.node_list[(len(deriv_tree.node_list)-1-2**n)-i].right.value))
    
    return deriv_tree.node_list[0].value

It should be noted that a person more clever than I would use the depth characteristic of the nodes to iterate in a less direct way. For example, to compute the price of the derivative, we need to iterate over the last 2^n elements of the tree, and then work our way up from the 2nd to last layer up. There is certainly a clean way to do this using node depth, but functional is better than not.

In [219]:
import timeit

In [220]:
%timeit BAMP_Tree_Opt_Price(4,.25,5,2,0.5,10)

10000 loops, best of 3: 79.4 µs per loop


In [221]:
BAMP_Tree_Opt_Price(4,.25,5,2,0.5,3)

2.3040000000000003

In [226]:
%timeit BAMP_MC_Opt_Price(4,.25,5,2,0.5,3,10000)

1 loop, best of 3: 578 ms per loop


In [229]:
BAMP_MC_Opt_Price(4,.25,5,2,0.5,3,10000)

2.2526976000000531

In [230]:
BAMP_MC_Opt_Price(4,.25,5,2,0.5,3,1000000)

2.2985410560085913

The tree method is MUCH faster and more accurate, to get similar accuracy with MC we need to run about a million trials, which is quite slow. It should be pointed out that this is only true for European options, for path dependent options flexibility will play a huge role, but that's a story for another day.